In [2]:
#webscraping here

import pandas as pd
import praw

reddit = praw.Reddit(client_id="69xO_MRhRBIQGprtVEZIbA",
    client_secret="GgPpkwrp_BPNduxYQ-dOcxfmrykfhQ",
    user_agent="test")


subreddit = reddit.subreddit("scu")

posts_dict = {"Body": [], "ID": [], "Ups": []}

for post in subreddit.hot(limit=1000):
    # Title and body of each post
    title = post.title
    text = post.selftext
    body = title + "\n" + text
    posts_dict['Body'].append(body)
     
    # Unique ID of each post
    posts_dict["ID"].append(post.id)
     
    # The score of a post
    posts_dict["Ups"].append(post.score)

top_posts = pd.DataFrame(posts_dict)
top_posts.to_csv("Top Posts.csv", index=True)
display(top_posts)



Version 7.7.0 of praw is outdated. Version 7.7.1 was released Tuesday July 11, 2023.


,Body,ID,Ups
0,Recruitments for Studies/Job Opportunities & O...,oouus2,17
1,Advice Paper for Incoming COEN/CS Students\nHi...,xkfy3u,49
2,Robotics and Automation in SCU\nI am from Asia...,1dbiqsj,1
3,"CRE QUESTION\nHello,\n\nI’ve took AP Calculus ...",1dbe0b2,2
4,Entire Townhome Available for Sublet//1st Quar...,1db2tb3,1
...,...,...,...
968,Freshman dorms\nHow hard is it to get into Fin...,kxpxqv,4
969,(Open to all Bay Area Residents) 1 Min Survey ...,kwq8y5,6
970,Hackathon Opportunity For SCU Students (All Ma...,ku4vj1,11
971,"Calling all engineering, computer science, and...",ks6uiv,3


In [3]:

id_list = top_posts["ID"].tolist()
#id_list = ["1d5yi68", "1cogl0"]
comments = pd.DataFrame()
comment_list = []
for i in id_list:                      #indicates start of the subission id processing
    submission = reddit.submission(i)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        if(comment.author == "AutoModerator"):
            continue
        else:
            comment_list.append(comment.__dict__)

comments_df = pd.DataFrame(comment_list)

cleaned_df = comments_df.filter(['_submission','id','author', 'body', 'ups'], axis=1)

cleaned_df.to_csv(f'comments.csv')

display(cleaned_df)

,_submission,id,author,body,ups
0,xkfy3u,ipxm05c,thepronoobkq,10/10 guide thx for teaching me scheme lol,1
1,xkfy3u,ipgi6cg,missionbmw,Shoutout to the best peer advisor ever!,5
2,xkfy3u,iptfalj,iSezdis,That's outstanding. Thanks for putting all the...,6
3,xkfy3u,itjbz9a,AcademicRevolution,This man is goated listen to his advice,4
4,xkfy3u,jmydfpc,lawfulpossum,Thank you so much!! So helpful :),5
...,...,...,...,...,...
4022,kogufc,l5so978,DifferentTax7203,"Old thread, but I’m a junior at SCU now and be...",1
4023,kogufc,ghqz2z7,None,[removed],1
4024,kogufc,gsiqso6,None,"I would love to, but I am a beginner too.\n\nA...",1
4025,kogufc,ghr5bgt,RemindMeBot,I will be messaging you in 4 days on [**2021-0...,1


In [15]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize

nltk.download('punkt')


tokenizer = RegexpTokenizer(r'\w+')

dorm_names = ['graham', 'finn', 'sanfilippo', 'campisi', 'mclaughlin-walsh', 'casa italiana', 'sobrato', 'swig', 'flip', 'camp', 'casa', 'walsh', 'university villas', 'nobili']
dorm_columns = ['graham', 'finn', 'sanfilippo', 'campisi', 'mclaughlin_walsh', 'casa_italiana', 'sobrato', 'swig', 'flip', 'camp', 'casa', 'walsh', 'university_villas', 'nobili']


# Function to extract dorm names from comments
def dorm_appears(comment, dorm):
    comment_lower = comment.lower()
    tokens = tokenizer.tokenize(comment_lower)
    return dorm in tokens

# Check if at least one dorm is found in the comment
def any_dorm_found(row):
    for dorm in dorm_names:
        dorm_column = dorm.replace(" ", "_").replace("-", "_") 
        if row[dorm_column]:
            return True
    return False

for dorm in dorm_names:
    dorm_column = dorm.replace(" ", "_").replace("-", "_")  # Ensure valid column names
    cleaned_df[dorm_column] = cleaned_df['body'].apply(dorm_appears, dorm=dorm)

# Remove rows where no dorms  found
cleaned_df = cleaned_df[cleaned_df.apply(any_dorm_found, axis=1)]


# Splitting comments with more than one dorm
new_rows = []
for col, row in cleaned_df.iterrows():
    sentences = sent_tokenize(row['body'])
    if sum(row[dorm_columns]) > 1:
        for sentence in sentences:
            new_row = row.copy()
            new_row['body'] = sentence
            new_rows.append(new_row)
    else:
        new_rows.append(row)

new_df = pd.DataFrame(new_rows)

# After new rows added re check sentences for dorm names
for dorm in dorm_names:
    dorm_column = dorm.replace(" ", "_").replace("-", "_")  # Ensure valid column names
    new_df[dorm_column] = new_df['body'].apply(dorm_appears, dorm=dorm)

# Remove rows where no dorms found
new_df = new_df[new_df.apply(any_dorm_found, axis=1)]


new_df.to_csv(f'comments_cleaned.csv')

display(new_df)
display(cleaned_df)

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


,_submission,id,author,body,ups,graham,finn,sanfilippo,campisi,mclaughlin_walsh,casa_italiana,sobrato,swig,flip,camp,casa,walsh,university_villas,nobili
48,1d5yi68,l6ostw4,Ch8ngus,Mo clue about being trans but graham and Finn ...,6,True,True,False,False,False,False,False,False,False,False,False,False,False,False
49,1d5yi68,l6oy0fr,5torm,Swig is “sustainability” and Casa is “Italian”).,4,False,False,False,False,False,False,False,True,False,False,True,False,False,False
51,1d5yi68,l6seh8v,Affectionate_Dog_600,"Swig, you will find it very inclusive there",-1,False,False,False,False,False,False,False,True,False,False,False,False,False,False
126,1cqr2oq,l576uig,Skicrazy85,"Each dorm has its own personality, I was in So...",2,False,False,False,False,False,False,True,True,False,False,False,False,False,False
163,1cm4xod,l36tyz2,Magellan_8888,It will be quite easy meeting people. I’m curr...,4,False,False,False,False,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,m8i6px,grojo66,SCUSKU,Congrats on the scholarship! That in and of it...,3,False,False,False,False,False,False,False,True,False,False,False,False,False,False
4014,kxpxqv,gjbwypo,iEshkker,Graham is pretty hard to get into mostly becau...,3,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4016,kxpxqv,gjenjk9,Bronco687,Swig fall 2021 bouta be crazy as fuck,1,False,False,False,False,False,False,False,True,False,False,False,False,False,False
4017,kxpxqv,gjdsg09,waffleseeds,Well this is my freshman year. I got into Dunn...,1,True,False,False,False,False,False,False,False,False,False,False,False,False,False


,_submission,id,author,body,ups,graham,finn,sanfilippo,campisi,mclaughlin_walsh,casa_italiana,sobrato,swig,flip,camp,casa,walsh,university_villas,nobili
48,1d5yi68,l6ostw4,Ch8ngus,Mo clue about being trans but graham and Finn ...,6,True,True,False,False,False,False,False,False,False,False,False,False,False,False
49,1d5yi68,l6oy0fr,5torm,There is no “best” dorm for LGBT people. The o...,4,False,False,False,False,False,False,False,True,False,False,True,False,False,False
51,1d5yi68,l6seh8v,Affectionate_Dog_600,"Swig, you will find it very inclusive there",-1,False,False,False,False,False,False,False,True,False,False,False,False,False,False
126,1cqr2oq,l576uig,Skicrazy85,"I can't speak to your admission chances, once ...",2,False,False,False,False,False,False,True,True,False,False,False,False,False,False
163,1cm4xod,l36tyz2,Magellan_8888,It will be quite easy meeting people. I’m curr...,4,False,False,False,False,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,m8i6px,grojo66,SCUSKU,Congrats on the scholarship! That in and of it...,3,False,False,False,False,False,False,False,True,False,False,False,False,False,False
4014,kxpxqv,gjbwypo,iEshkker,Graham is pretty hard to get into mostly becau...,3,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4016,kxpxqv,gjenjk9,Bronco687,Swig fall 2021 bouta be crazy as fuck,1,False,False,False,False,False,False,False,True,False,False,False,False,False,False
4017,kxpxqv,gjdsg09,waffleseeds,Well this is my freshman year. I got into Dunn...,1,True,False,False,False,False,False,False,False,False,False,False,False,False,False


In [5]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess_and_tokenize(text):
    # Clean the text as previously defined
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    cleaned_text = " ".join(new_text)
    
    # Tokenize and ensure consistent input size with padding and truncation
    return tokenizer(cleaned_text, padding="max_length", max_length=128, truncation=True, return_tensors='pt')
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

def sent_analyze(text):
    try:
        encoded_input = preprocess_and_tokenize(text)
        output = model(**encoded_input)
        scores = output.logits.detach().numpy()
        scores = softmax(scores, axis=1)
        return scores[0]  # Return the first batch as you're processing texts one by one
    except Exception as e:
        print(f"Error processing text: {text}")
        print(e)
        return [0, 0, 0]  # Return a default value in case of error


x = sent_analyze("I love the food at Graham Hall")
print(x[0], "," , x[1], "," , x[2]) 

ModuleNotFoundError: No module named 'transformers'

In [ ]:

import pandas as pd
from scipy.special import softmax

# Assuming the rest of your setup (imports and function definitions like sent_analyze) is correct and above this code block

# Apply the sentiment analysis function to each row in the 'Body' column
top_posts['sentiment'] = top_posts['Body'].apply(sent_analyze)

# Create new columns for negative, neutral, and positive scores by extracting the respective indices from the sentiment list
top_posts['neg'] = top_posts['sentiment'].apply(lambda x: x[0])
top_posts['neu'] = top_posts['sentiment'].apply(lambda x: x[1])
top_posts['pos'] = top_posts['sentiment'].apply(lambda x: x[2])

# Optionally, you can drop the 'sentiment' column if it's no longer needed
top_posts.drop('sentiment', axis=1, inplace=True)

# View the DataFrame to confirm the new columns are added correctly
display(top_posts)

,Body,ID,Ups,graham,finn,sanfilippo,campisi,mclaughlin_walsh,casa_italiana,sobrato,swig,flip,camp,casa,walsh,university_villas,nobili,neg,neu,pos
38,What are my chances?\nHey everyone!\n\nI'm cur...,1cp9tqh,4,False,False,False,False,False,False,False,False,False,True,False,False,False,False,0.010929,0.473358,0.515713
49,Graham/Finn\nDo incoming freshman have a chanc...,1cmhmzn,3,True,True,False,False,False,False,False,False,False,False,False,False,False,False,0.014901,0.957815,0.027284
67,"Transfer Housing Information\nHello, I'm a tra...",1chxhvo,3,False,True,False,False,False,False,False,False,False,False,False,False,False,False,0.018936,0.893320,0.087744
76,Orientation Sessions are open for registration...,1cd3a17,7,False,False,False,False,False,False,False,True,False,False,False,False,False,False,0.006647,0.892695,0.100658
80,Dunne bathroom updated?\nI am about to submit ...,1camxec,1,False,False,False,False,False,False,False,False,False,False,False,True,False,False,0.090354,0.853742,0.055904
101,"Santa Clara University or UC Santa Cruz, Which...",1c22i9j,2,False,False,False,False,False,False,False,False,True,False,False,False,False,False,0.012864,0.678406,0.308730
114,Complaining about Swig & Swig Residents until ...,1byptue,2,False,False,False,False,False,False,False,True,False,False,False,False,False,False,0.382792,0.564780,0.052428
156,Housing at SCU\nI was wondering how housing wo...,1bmxuwd,2,True,False,False,False,False,False,False,False,False,False,True,False,False,False,0.063373,0.844904,0.091723
183,If you’re a first year honors student are you ...,1bh9dfi,2,False,False,False,False,False,False,False,True,False,False,False,False,False,False,0.046505,0.909442,0.044052
186,"Rank the dorms\nPlease rank\n\nCasa, campesi, ...",1bf62mi,7,False,False,True,False,False,False,True,False,False,False,True,False,False,False,0.025741,0.808154,0.166104
